### Import Libraries

In [ ]:
import pandas as pd
import pathlib
from joblib import load

import sys
sys.path.append("../utils")
from split_utils import get_features_data
from train_utils import get_dataset
from evaluate_utils import model_score

### Load Necessary Data

In [ ]:
# specify results directory
results_dir = pathlib.Path("evaluations/")
f1_scores_dir = pathlib.Path(f"{results_dir}/F1_scores/")
f1_scores_dir.mkdir(parents=True, exist_ok=True)

# load features data from indexes and features dataframe
data_split_path = pathlib.Path("../1.split_data/indexes/data_split_indexes.tsv")
data_split_indexes = pd.read_csv(data_split_path, sep="\t", index_col=0)
features_dataframe_path = pathlib.Path("../0.download_data/data/training_data.csv.gz")
features_dataframe = get_features_data(features_dataframe_path)

### Evaluate Each Model on Each Dataset

In [ ]:
# directory to load the models from
models_dir = pathlib.Path("../2.train_model/models/")

# iterate through models
for model_path in models_dir.iterdir():
    model = load(model_path)
    model_name = model_path.name.replace("log_reg_","").replace(".joblib","")
    
    # iterate through label datasets
    for label in data_split_indexes["label"].unique():
        print(f"Evaluating {model_name} on dataset {label}")
        # load dataset (train, test, etc)
        data = get_dataset(features_dataframe, data_split_indexes, label)
        # path to save scores to
        score_save_path = pathlib.Path(f"{f1_scores_dir}/{model_name}_{label}.tsv")
        
        score = model_score(model, data)
        score = score.T.reset_index()
        score.columns = ["Phenotypic_Class", "F1_Score"]
        score.to_csv(score_save_path, sep="\t")